# Approximation of Hodgkin-Huxley equations

## XPAUT Code

The code to simulate the system in XPAUT is shown below.

    # Simplified Hodgkin and Huxley equations simulation
    # Created by Lucas van Dijk.

    par C=20
    par V_na=120, V_k=-84, V_l=-60
    par v1=-1.2, v2=18, v3=8, v4=30

    par g_na=4.4, g_k=8, g_l=2
    par I_app=0
    par phi_n=25

    m_inf(V)=(1/2)*(1+tanh((V-v1)/v2))
    h_0 = 0.9

    n_inf(V)=(1/2)*(1+tanh((V-v3)/v4))
    tau_n(V)=phi_n*cosh((V-v3)/(2*v4))


    dn/dt = (n_inf(V)-n)/(tau_n(V))
    dV/dt = ((-g_na*(m_inf(V))^3*h_0*(V-V_na)) - (g_k*n^4*(V-V_k)) - (g_l*(V-V_l)) + I_app)/C

    init V=60, n=-0.6

    done
    
## Nullclines

![System Nullclines](nullclines.png)

## Trajectories

![A few example trajectories](trajectories.png)

## Time series plot

Initial conditions: $V = 60$ mV, $n = -0.6$

![Time series plot](time-simulation.png)




## Stability Analysis with Python

In [1]:
%matplotlib inline
from sympy import Function, Derivative, symbols, tanh, cosh, Symbol, init_printing, Eq
import numpy as np


init_printing()

ImportError: No module named 'sympy'

In [ ]:
# System parameters

v1 = -1.2  # mV
v2 = 18  # mV
v3 = 8  # mV
v4 = 30  # mV

V_k = -84  # mV
V_na = 120  # mV
V_l = -60  # mV

g_k = 8  # mS/cm^2
g_na = 4.4  # ms/cm^2
g_l = 2  # mS/cm^2

C = 20  # uF/cm^2
phi_n = 25  # ms
h0 = 0.9


In [ ]:
t = Symbol('t')
V = Function('V')(t)
n = Function('n')(t)

### Define functions

In [ ]:
m_inf = (1 + tanh((V - v1)/v2))/2
m_inf

In [ ]:
n_inf = (1 + tanh((V - v3)/v4))/2
n_inf


In [ ]:
tau_n = phi_n*cosh((V - v3)/(2*v4))
tau_n


### Differential Equations

In [ ]:
# Differential equation for V(t)
eq1 = Eq(C*V.diff(t), -g_na * (m_inf)**3 * h0 * (V - V_na) - g_k* n**4 * (V - V_k) - g_l*(V - V_l))
eq1

In [ ]:
# Differential Equation for n(t)
eq2 = Eq(n.diff(t), (n_inf - n)/tau_n)
eq2

### Functions used to calculate the Jacobian

In [ ]:
f = (-g_na * (m_inf)**3 * h0 * (V - V_na) - g_k* n**4 * (V - V_k) - g_l*(V - V_l))/C
g = (n_inf - n)/tau_n

In [ ]:
f.diff(V)

In [ ]:
f.diff(n)

In [ ]:
g.diff(V)

In [ ]:
g.diff(n)

### Calculate the Jacobian for all steady states

And also determine its eigenvalues.

In [ ]:
steady_states = [
    (0, -60),
    (0.5, 10),
    (0.6, 14.5)
]

for steady_state in steady_states:
    j11 = float(f.diff(V).evalf(subs={V: steady_state[1], n: steady_state[0]}))
    j12 = float(f.diff(n).evalf(subs={V: steady_state[1], n: steady_state[0]}))
    j21 = float(g.diff(V).evalf(subs={V: steady_state[1], n: steady_state[0]}))
    j22 = float(g.diff(n).evalf(subs={V: steady_state[1], n: steady_state[0]}))
    
    jacobian = np.array([
        [j11, j12],
        [j21, j22]
    ])
    
    print("Steady state at n={}, V={}".format(*steady_state))
    print("="*20)
    print("[-] Jacobian:")
    print(jacobian)
    
    print("[-] Eigenvalues")
    eigen_values = np.linalg.eig(jacobian)
    print(eigen_values[0])

Only the first steady state is stable (for both eigenvalues the real part is negative).